In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import pickle

In [34]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [35]:
## data analysis
## drop irrelevant features

df.drop(columns=['RowNumber','CustomerId','Surname'],axis=1,inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [36]:
## Feature Selection

X = df.drop('Exited',axis=1)
y = df['Exited']

In [37]:
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


In [38]:
## extract numerical and categorical features

categorical_features = [feature for feature in X.columns if df[feature].dtype=='O']
categorical_features

['Geography', 'Gender']

In [39]:
numerical_features = [feature for feature in X.columns if feature not in categorical_features]
numerical_features

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [40]:
[X[feature].value_counts() for feature in categorical_features]

[Geography
 France     5014
 Germany    2509
 Spain      2477
 Name: count, dtype: int64,
 Gender
 Male      5457
 Female    4543
 Name: count, dtype: int64]

In [41]:
categorical_features.remove('Gender')

In [42]:
##Here we apply label encoder for Gender and we apply OneHotEncoder for Geography,becaues if we apply labelencoder
## to geography it assigns as 0,1,2 model thinks that 2 is greater than 1.

from sklearn.preprocessing import LabelEncoder

gender_label_encoder = LabelEncoder()

X['Gender'] = gender_label_encoder.fit_transform(X['Gender'])

In [43]:
with open('gender_label_encoder.pkl','wb') as file:
    pickle.dump(gender_label_encoder,file)

In [44]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0.00,1,1,1,101348.88
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58
2,502,France,0,42,8,159660.80,3,1,0,113931.57
3,699,France,0,39,1,0.00,2,0,0,93826.63
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10


In [45]:
numerical_features

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [46]:
categorical_features

['Geography']

In [47]:
## Here we converted gender into numerical values for the model training
## Now we will convert geography in numericals with the help of OneHotEncoder
## Along with that we will convert numerical in the standardization form
## we will do this with the help of columntransformer

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

In [48]:
transformer = ColumnTransformer(
    [
        ('scaler',StandardScaler(),numerical_features),
        ('encoding',OneHotEncoder(drop='first'),categorical_features)
    ],
remainder='passthrough')

In [49]:
transformed_data = transformer.fit_transform(X)

In [50]:
with open('transformer.pkl','wb') as file:
    pickle.dump(transformer,file)

In [51]:
pd.DataFrame(transformed_data)

,0,1,2,3,4,5,6,7,8,9,10
0,-0.326221,0.293517,-1.041760,-1.225848,-0.911583,0.646092,0.970243,0.021886,0.0,0.0,0.0
1,-0.440036,0.198164,-1.387538,0.117350,-0.911583,-1.547768,0.970243,0.216534,0.0,1.0,0.0
2,-1.536794,0.293517,1.032908,1.333053,2.527057,0.646092,-1.030670,0.240687,0.0,0.0,0.0
3,0.501521,0.007457,-1.387538,-1.225848,0.807737,-1.547768,-1.030670,-0.108918,0.0,0.0,0.0
4,2.063884,0.388871,-1.041760,0.785728,-0.911583,0.646092,0.970243,-0.365276,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
9995,1.246488,0.007457,-0.004426,-1.225848,0.807737,0.646092,-1.030670,-0.066419,0.0,0.0,1.0
9996,-1.391939,-0.373958,1.724464,-0.306379,-0.911583,0.646092,0.970243,0.027988,0.0,0.0,1.0
9997,0.604988,-0.278604,0.687130,-1.225848,-0.911583,-1.547768,0.970243,-1.008643,0.0,0.0,0.0
9998,1.256835,0.293517,-0.695982,-0.022608,0.807737,0.646092,-1.030670,-0.125231,1.0,0.0,1.0


In [56]:
transformer.get_feature_names_out

<bound method ColumnTransformer.get_feature_names_out of ColumnTransformer(remainder='passthrough',
                  transformers=[('scaler', StandardScaler(),
                                 ['CreditScore', 'Age', 'Tenure', 'Balance',
                                  'NumOfProducts', 'HasCrCard',
                                  'IsActiveMember', 'EstimatedSalary']),
                                ('encoding', OneHotEncoder(drop='first'),
                                 ['Geography'])])>

In [57]:
## now our data set is completely ready for model training, lets split the data

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(transformed_data,y)

In [58]:
transformed_data.shape,X_train.shape

((10000, 11), (7500, 11))

In [61]:
## let's build our ann model

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential(
    [
        Dense(64, activation='relu',input_shape=(X_train.shape[1],)), ## HL1
        Dense(32,activation='relu'), ##HL2 , Here we dont use more hl's bacuase model would be overfit
        Dense(1,activation='sigmoid') ## output layer
    ]
)

In [64]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 64)                768       
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2881 (11.25 KB)
Trainable params: 2881 (11.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [66]:
import tensorflow
adam = tensorflow.keras.optimizers.Adam(learning_rate=0.01)


In [67]:
model.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy'])

In [69]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

#### Set up the Tensorboard

In [70]:
log_dir = "log/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [71]:
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

#### Set up the EarlyStopping

In [74]:
import warnings
warnings.filterwarnings('ignore')

In [75]:
earlystopping_callback = EarlyStopping(monitor='val_loss', patience=10,restore_best_weights=True)

### Train the Model

In [76]:
model.fit(X_train,y_train, validation_data=(X_test,y_test),epochs=50,
          callbacks=[tensorflow_callback,earlystopping_callback])

Epoch 1/50
235/235 [==============================] - 1s 6ms/step - loss: 0.2341 - accuracy: 0.9000 - val_loss: 0.5366 - val_accuracy: 0.8412
Epoch 2/50
235/235 [==============================] - 1s 5ms/step - loss: 0.2310 - accuracy: 0.9015 - val_loss: 0.5435 - val_accuracy: 0.8480
Epoch 3/50
235/235 [==============================] - 1s 5ms/step - loss: 0.2235 - accuracy: 0.9025 - val_loss: 0.5596 - val_accuracy: 0.8372
Epoch 4/50
235/235 [==============================] - 1s 4ms/step - loss: 0.2269 - accuracy: 0.9028 - val_loss: 0.5812 - val_accuracy: 0.8436
Epoch 5/50
235/235 [==============================] - 1s 5ms/step - loss: 0.2245 - accuracy: 0.9053 - val_loss: 0.5807 - val_accuracy: 0.8396
Epoch 6/50
235/235 [==============================] - 1s 5ms/step - loss: 0.2197 - accuracy: 0.9073 - val_loss: 0.5661 - val_accuracy: 0.8436
Epoch 7/50
235/235 [==============================] - 1s 4ms/step - loss: 0.2245 - accuracy: 0.9043 - val_loss: 0.5882 - val_accuracy: 0.8332
Epoch 

In [77]:
model.save('model.h5')

In [79]:
%load_ext tensorboard

In [85]:
%tensorboard --logdir log/fit --port 6007